# Clement's Notes
Here, we will attempt to call Ollama from Python code and have it run locally. We will use the locallally running Ollama to generate text based on a prompt.

# Welcome to your first assignment!

Instructions are below. Please give this a try, and look in the solutions folder if you get stuck (or feel free to ask me!)

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Just before we get to the assignment --</h2>
            <span style="color:#f71;">I thought I'd take a second to point you at this page of useful resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

# HOMEWORK EXERCISE ASSIGNMENT

Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

You'll be able to use this technique for all subsequent projects if you'd prefer not to use paid APIs.

**Benefits:**
1. No API charges - open-source
2. Data doesn't leave your box

**Disadvantages:**
1. Significantly less power than Frontier Model

## Recap on installation of Ollama

Simply visit [ollama.com](https://ollama.com) and install!

Once complete, the ollama server should already be running locally.  
If you visit:  
[http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter `ollama serve`  
And in another Terminal (Mac) or Powershell (Windows), enter `ollama pull llama3.2`  
Then try [http://localhost:11434/](http://localhost:11434/) again.

If Ollama is slow on your machine, try using `llama3.2:1b` as an alternative. Run `ollama pull llama3.2:1b` from a Terminal or Powershell, and change the code below from `MODEL = "llama3.2"` to `MODEL = "llama3.2:1b"`

In [1]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [2]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [3]:
# Create a messages list using the same format that we used for OpenAI

messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]

In [4]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [5]:
# Let's just make sure the model is loaded

!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 34bb5ab01051: 100% ▕██████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [6]:
# If this doesn't work for any reason, try the 2 versions in the following cells
# And double check the instructions in the 'Recap on installation of Ollama' at the top of this lab
# And if none of that works - contact me!

response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

Generative AI has numerous business applications across various industries. Here are some examples:

1. **Content Creation**: Generative AI can be used to generate high-quality content such as articles, social media posts, product descriptions, and more. This can help businesses save time and resources while maintaining consistency in their content.
2. **Product Design and Development**: Generative AI can aid in the design of new products, including 3D models, user interfaces, and even entire product lines. This can speed up the product development process and reduce costs.
3. **Marketing and Advertising**: Generative AI can be used to generate personalized marketing campaigns, social media ads, and even entire marketing strategies. It can also help analyze consumer behavior and preferences.
4. **Customer Service Chatbots**: Generative AI can create conversational chatbots that mimic human-like interactions, helping businesses provide 24/7 customer support and improve customer satisfac

# Introducing the ollama package

And now we'll do the same thing, but using the elegant ollama python package instead of a direct HTTP call.

Under the hood, it's making the same call as above to the ollama server running at localhost:11434

In [7]:
import ollama

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

Generative AI has numerous business applications across various industries, including:

1. **Content Creation**: Generate high-quality content such as articles, social media posts, product descriptions, and more, using natural language processing (NLP) and machine learning algorithms.
2. **Product Design**: Create 3D models, product prototypes, and visualizations for product design, architecture, and engineering applications.
3. **Marketing and Advertising**: Generate personalized marketing campaigns, ad copy, and social media content that resonates with target audiences.
4. **Customer Service Chatbots**: Develop chatbots that can respond to customer inquiries, provide support, and offer recommendations.
5. **Data Analysis and Insights**: Use Generative AI to analyze large datasets, identify patterns, and generate predictive models for business decision-making.
6. **Automated Writing Tools**: Generate reports, proposals, emails, and other documents with the help of AI-powered writing a

## Alternative approach - using OpenAI python library to connect to Ollama

In [ ]:
# There's actually an alternative approach that some people might prefer
# You can use the OpenAI client python library to call Ollama:

from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL,
    messages=messages
)

print(response.choices[0].message.content)

## Are you confused about why that works?

It seems strange, right? We just used OpenAI code to call Ollama?? What's going on?!

Here's the scoop:

The python class `OpenAI` is simply code written by OpenAI engineers that makes calls over the internet to an endpoint.  

When you call `openai.chat.completions.create()`, this python code just makes a web request to the following url: "https://api.openai.com/v1/chat/completions"

Code like this is known as a "client library" - it's just wrapper code that runs on your machine to make web requests. The actual power of GPT is running on OpenAI's cloud behind this API, not on your computer!

OpenAI was so popular, that lots of other AI providers provided identical web endpoints, so you could use the same approach.

So Ollama has an endpoint running on your local box at http://localhost:11434/v1/chat/completions  
And in week 2 we'll discover that lots of other providers do this too, including Gemini and DeepSeek.

And then the team at OpenAI had a great idea: they can extend their client library so you can specify a different 'base url', and use their library to call any compatible API.

That's it!

So when you say: `ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')`  
Then this will make the same endpoint calls, but to Ollama instead of OpenAI.

## Also trying the amazing reasoning model DeepSeek

Here we use the version of DeepSeek-reasoner that's been distilled to 1.5B.  
This is actually a 1.5B variant of Qwen that has been fine-tuned using synethic data generated by Deepseek R1.

Other sizes of DeepSeek are [here](https://ollama.com/library/deepseek-r1) all the way up to the full 671B parameter version, which would use up 404GB of your drive and is far too large for most!

In [ ]:
!ollama pull deepseek-r1:1.5b

In [ ]:
# This may take a few minutes to run! You should then see a fascinating "thinking" trace inside <think> tags, followed by some decent definitions

response = ollama_via_openai.chat.completions.create(
    model="deepseek-r1:1.5b",
    messages=[{"role": "user", "content": "Please give definitions of some core concepts behind LLMs: a neural network, attention and the transformer"}]
)

print(response.choices[0].message.content)

# NOW the exercise for you

Take the code from day1 and incorporate it here, to build a website summarizer that uses Llama 3.2 running locally instead of OpenAI; use either of the above approaches.